# Get data about teams and players

In [130]:
import requests
import pandas as pd
import os

In [131]:
pd.set_option('display.max_columns', None)

## Get basic player and team details

In [132]:
response = requests.get('https://en.fantasy.allsvenskan.se/api/bootstrap-static/')
data = response.json();
player_data = pd.DataFrame(data['elements'])
player_data.to_csv('data/player_details_raw.csv', index=False)
player_data.sample(10)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,own_goals,attacking_bonus,defending_bonus,crosses,key_passes,big_chances_created,clearances_blocks_interceptions,recoveries
407,100.0,100.0,432539,0,0,0,0,0,4,2.4,2.4,0,Ryan,2.4,430,False,,2021-07-10T07:16:30.998925Z,50,432539.jpg,2.7,Moon,0.6,False,None,a,13,6810,32,371,1,213,6,0.5,6.4,Moon,579,3,0,3,10,0,0,2,0,0,0,0,0,1,3,0,4,20
310,NaN,NaN,70064,0,0,-2,2,0,3,2.6,2.6,0,Romario,2.6,266,False,,None,63,70064.jpg,3.6,Pereira Sipião,2.3,False,None,a,10,2594,54,224,5,528,1,0.4,8.6,Romario,1339,1,0,5,16,0,0,2,0,0,0,1,15,2,7,0,41,125
201,100.0,100.0,61768,0,0,-1,1,0,1,0.0,0.0,0,Tommy,0.0,167,False,,2021-04-12T07:53:52.476708Z,59,61768.jpg,2.0,Vaiho,0.7,False,None,a,7,1406,2,134,1,335,1,0.0,0.3,Vaiho,64,0,0,0,2,0,0,0,0,2,0,0,1,0,0,0,1,8
432,100.0,100.0,523993,0,0,-1,1,0,2,0.6,0.6,0,Herman,0.6,368,False,,2021-04-12T07:28:51.096520Z,44,523993.jpg,1.0,Johansson,0.1,False,None,a,14,2360,3,4,0,19,0,0.1,0.7,Johansson,36,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
385,NaN,NaN,506220,0,0,-1,1,0,2,0.0,0.0,0,Nils,0.0,330,False,,None,39,506220.jpg,0.0,Bertilsson,1.2,False,None,a,13,6810,0,95,0,258,0,0.0,0.0,Bertilsson,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
64,NaN,NaN,533595,0,0,-1,1,0,4,0.2,0.2,0,Jusef,0.2,469,False,,None,44,533595.jpg,1.0,Erabi,0.0,False,None,a,2,1987,1,3,0,2,0,0.0,0.2,Erabi,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
435,0.0,0.0,467607,0,0,-4,4,0,4,0.0,0.0,0,Taylor,0.0,371,False,Rehabtränar. Tillbaka i spel: 1-2 veckor,2021-06-28T08:53:42.543973Z,41,467607.jpg,1.0,Silverholt,1.8,False,None,i,14,2360,3,371,0,1679,1,0.0,0.7,Silverholt,40,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2
235,NaN,NaN,484393,0,0,-2,2,0,2,0.0,0.0,0,Niclas,0.0,200,False,,None,38,484393.jpg,1.0,Bergmark,2.4,False,None,a,8,2596,2,442,8,576,3,0.0,0.5,Bergmark,23,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,1
425,NaN,NaN,506231,0,0,-1,1,0,3,0.0,0.0,0,Adam,0.0,361,False,,None,44,506231.jpg,1.0,Petersson,0.2,False,None,a,14,2360,5,7,0,22,0,0.0,1.1,Petersson,48,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,6,1
434,NaN,NaN,479403,0,0,-4,4,1,4,1.6,1.6,0,Mamudu,1.6,370,False,,None,51,479403.jpg,2.2,Moro,2.6,False,None,a,14,2360,28,89,0,1562,2,0.3,5.5,Moro,292,3,0,0,3,0,0,2,0,0,0,0,1,1,3,0,2,22


In [133]:
teams_data = pd.DataFrame(data['teams'])
teams_data.to_csv('data/teams_raw.csv', index=False)
teams_data.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win
0,993,0,None,1,0,Malmö FF,0,0,0,MFF,None,None,False,0
1,1987,0,None,2,0,Hammarby,0,0,0,HAM,None,None,False,0
2,3056,0,None,3,0,IK Sirius,0,0,0,IKS,None,None,False,0
3,244,0,None,4,0,IFK Göteborg,0,0,0,GBG,None,None,False,0
4,375,0,None,5,0,IF Elfsborg,0,0,0,IFE,None,None,False,0


## Get all players game history

In [134]:
for index, player in player_data.iterrows():
  player_id = player['id']
  player_name = f"{player['first_name']}_{player['second_name']}".replace(' ', '_')
  player = requests.get(f'https://fantasy.allsvenskan.se/api/element-summary/{player_id}/')
  data = player.json()
  player_history = pd.DataFrame(data['history'])
  player_history.to_csv(f'data/player_history/{player_id}_{player_name}.csv', index=False)

## Merge all players game history to a single DataFrame

In [135]:
player_history_dir = os.fsencode('data/player_history')

players_history = pd.DataFrame()
for file in os.listdir(player_history_dir):
  filename = f'data/player_history/{os.fsdecode(file)}'
  
  try:
    file_data = pd.read_csv(filename, index_col=False, header=0)
  except pd.errors.EmptyDataError:
    file_data = pd.DataFrame()
  players_history = players_history.append(file_data)

players_history.sort_values(by=['element', 'round'], inplace=True)
players_history.to_csv('data/player_game_history.csv', index=False)

players_history.sample(5)

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,own_goals,attacking_bonus,defending_bonus,crosses,key_passes,big_chances_created,clearances_blocks_interceptions,recoveries,value,transfers_balance,selected,transfers_in,transfers_out
15,263,128,11,0,False,2021-08-23T17:00:00Z,NaN,NaN,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43,2,50,4,2
1,401,11,10,0,True,2021-04-18T13:00:00Z,0.0,1.0,2,43,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,3,3,50,-8,281,6,14
1,306,15,4,0,False,2021-04-19T17:00:00Z,2.0,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55,-3,35,0,3
7,351,57,12,3,False,2021-05-22T13:00:00Z,2.0,2.0,8,90,0,0,0,2,0,0,0,0,3,0,0,1,0,0,0,2,6,49,43,828,51,8
7,36,62,10,0,False,2021-05-23T13:00:00Z,2.0,1.0,8,29,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,4,4,48,-15,384,1,16


## Make a clean dataset of the interesting columns

In [144]:
player_game_history_clean = players_history[["element", "round", "opponent_team", "was_home", "total_points"]]
player_game_history_clean.sample(10)

,element,round,opponent_team,was_home,total_points
8,173,9,8,True,0
13,307,14,7,False,3
2,117,3,14,True,0
6,225,7,13,False,1
10,69,11,7,False,1
6,366,7,6,True,0
6,170,7,2,False,0
14,268,15,15,True,2
4,393,5,8,False,2
5,298,6,1,True,4


## Set player id, player name and team names

In [147]:
# player_id
player_game_history_clean = player_game_history_clean.rename(columns={'element': 'player_id', 'opponent_team': 'opponent_team_id'})

# player_name
player_data["player_name"] = player_data["first_name"] + ' ' + player_data["second_name"]
player_data["player_name"]
player_game_history_clean["player_name"] = player_game_history_clean["player_id"].map(player_data.set_index("id")["player_name"])

# team_name
player_game_history_clean["team_id"] = player_game_history_clean["player_id"].map(player_data.set_index("id")["team"])
player_game_history_clean["team_name"] = player_game_history_clean["team_id"].map(teams_data.set_index("id")["name"])

# opp_team_name
player_game_history_clean["opponent_team_name"] = player_game_history_clean["opponent_team_id"].map(teams_data.set_index("id")["name"])

# Reorder columns
player_game_history_clean_final = player_game_history_clean[["round", "player_name", "team_name", "opponent_team_name", "was_home", "total_points"]]
player_game_history_clean_final

,round,player_name,team_name,opponent_team_name,was_home,total_points
0,1,Johan Dahlin,Malmö FF,Hammarby,True,0
1,2,Johan Dahlin,Malmö FF,BK Häcken,False,0
2,3,Johan Dahlin,Malmö FF,Östersunds FK,True,0
3,4,Johan Dahlin,Malmö FF,Djurgården,False,0
4,5,Johan Dahlin,Malmö FF,Varbergs BoIS,True,0
...,...,...,...,...,...,...
1,16,Alfie Whiteman,Degerfors IF,Malmö FF,False,0
0,15,Karl Serrano,Degerfors IF,BK Häcken,True,0
1,16,Karl Serrano,Degerfors IF,Malmö FF,False,0
0,16,Nikola Djurdjic,Degerfors IF,Malmö FF,False,0


In [148]:
player_game_history_clean_final.to_csv('data/player_game_history_clean.csv')